<a href="https://colab.research.google.com/github/gsdev42/Medical-question-summarization-/blob/main/Fine_tuning_T_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers
!pip install -U datasets
!pip install tensorboard
!pip install sentencepiece
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.7 MB/s eta 0:00:00


In [ ]:
 !pip install h5py typing-extensions wheel

In [ ]:
import torch

from transformers import (
    T5Tokenizer,
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer
)
from datasets import load_dataset

In [ ]:
MODEL = 't5-small'
BATCH_SIZE = 48
NUM_PROCS = 16
EPOCHS = 10
OUT_DIR = 'results_t5small'
MAX_LENGTH = 256 # Maximum context length to consider while preparing dataset.

In [ ]:
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer
import os
import pandas as pd

In [ ]:
# Define paths to your datasets
train_data_path = "training_MeQSum.csv"
valid_data_path = "testing_MeQSum.csv"

# Function to load and convert CSV to Dataset
def load_and_convert_csv(file_path):
    try:
        df = pd.read_csv(file_path, encoding='ISO-8859-1')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='latin1')
    return Dataset.from_pandas(df)

# Load and convert datasets
train_dataset = load_and_convert_csv(train_data_path)
valid_dataset = load_and_convert_csv(valid_data_path)

In [ ]:
# Load the T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Preprocess the data
def preprocess_data(examples):
    inputs = examples['question']
    targets = examples['summary']
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=150, truncation=True, padding='max_length')

    model_inputs['labels'] = labels['input_ids']
    return model_inputs


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Apply preprocessing
tokenized_train = train_dataset.map(preprocess_data, batched=True)
tokenized_valid = valid_dataset.map(preprocess_data, batched=True)

# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained('t5-small')

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Define the output directory
output_directory = './results/t5_meqsum'

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

In [ ]:
# Load the T5 model
model = T5ForConditionalGeneration.from_pretrained(MODEL)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")


60,506,624 total parameters.
60,506,624 training parameters.


In [ ]:
# Define the output directory
output_directory = './results/t5_meqsum'
os.makedirs(output_directory, exist_ok=True)

# Define training arguments
EPOCHS = 3
BATCH_SIZE = 8

In [ ]:
training_args = TrainingArguments(
    output_dir=output_directory,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir=output_directory,
    logging_steps=10,
    evaluation_strategy='steps',
    save_steps=500,
    eval_steps=500,
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to='tensorboard',
    learning_rate=0.0001,
    fp16=False,
    dataloader_num_workers=0
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
)

In [ ]:
# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained(output_directory)
tokenizer.save_pretrained(output_directory)

Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss
500,0.178100,0.127550


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


('./results/t5_meqsum/tokenizer_config.json',
 './results/t5_meqsum/special_tokens_map.json',
 './results/t5_meqsum/spiece.model',
 './results/t5_meqsum/added_tokens.json')

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

output_directory = './results/t5_meqsum'

# Load the model and tokenizer
tokenizer = T5Tokenizer.from_pretrained(output_directory)
model = T5ForConditionalGeneration.from_pretrained(output_directory)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
ref_summary_1="""How can I find physician(s) or hospital(s) who specialize in pheochromocytomas?"""
ref_summary_2="""Where can I find information on mecp2 gene mutation and ppm-x syndrome?"""

In [ ]:
sample1="""I understand that you cannot provide opinions,nor suggest, any type of therpy.
My son (dob-[DATE]), was called back to the hospital four days after birth due to a abnormal PKU, which suggested adrenal hyperplasia.
After test the endocrine physians felt he did not have the condition. Several years later he began having severe headaches preceded by violent
vomiting epidodes that lasted for hours. When he was admitted to [LOCATION],  he was suffering from bleeding esphogal ulcers,  and severe
dehydration and, malnutrician. His urine (24 hr), blood plasma,& VMA,showed increased values of catacholomines.He also demostrated intermittent
hypertension, and tachcardia through out the day (he had been diagnoised with ADHD several years earlier). His MIGB clearly showed bifocal uptake
in both adrenal glands. The mri did not show tumors at that time ([DATE]. He has since been diagnoised with antral polys (biosphy's taken were misplaced,
 and the gastrologist would not redo the test), aberrant right subclavian artery,  esphogas pushed to the side, low oxygen intake, high levels of gastrin,
  cyclic vomiting syndrome, severe sinus attacks that come and go, without infection present.
His symptons that I have not already include heat intolerence, pallor, poor appetite with little weight gain, anxiety, tremor of hands, and stomach pain.
After his mri, which I was told would take about 45mins, took over 5 hrs. Since then we have been having no luck with finding doctors that will take my
son as a patient. Are you able to suggest a hospital that is knowlegable with pheochromocytomas? My son's father is mia, but during our 7 yrs together,
  he also suffered severe headaches, and intermittent hypertension. He never followed up with a physian. Any help would be appreciated. I do not know
  what to do, and I do not want my son to end up a statistic.  Last night his BB was 180/115, with HR-50, half hour later his BP was 68/44 with HR of 122.
   I fear he is going to go in shoc"""

sample2="""SUBJECT: ClinicalTrials.gov - Question - general information
MESSAGE: I have a [AGE] and a [AGE] boys who have been diagnosed with the mecp2 gene mutation and yesterday my neurologist was also commenting
 on the possibility of ppm-x I am very concerned that no-one in this field can he me. I have been seen durastic changes that have no changes
 like any of the other characteristics that have been reported.I am desperate to keep them safe and comfortable for the rest of life and could
 realy use help.  [NAME]   [CONTACT] please help me with any information it would be appreciated more than you know."""


In [ ]:
# Format the input text as required by the model
input_text = f"assign tag: {ref_summary_1} {sample1}"

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors='pt', truncation=True, padding='max_length', max_length=512).input_ids
input_ids = input_ids.to(device)

In [ ]:
# Generate the prediction
outputs = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)

# Decode the prediction
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Tags:", prediction)

Generated Tags: What are the treatments for pheochromocytomas?


In [ ]:
# Format the input text as required by the model
input_text = f"assign tag: {ref_summary_2} {sample2}"

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors='pt', truncation=True, padding='max_length', max_length=512).input_ids
input_ids = input_ids.to(device)

In [ ]:
# Generate the prediction
outputs = model.generate(input_ids, max_length=150, num_beams=4, early_stopping=True)

# Decode the prediction
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Generated Tags:", prediction)

Generated Tags: Where can I find information on mecp2 gene mutation and ppm-x syndrome?


In [ ]:
#generated summaries
summ1="""Where can I find information on mecp2 gene mutation and ppm-x syndrome?"""
summ2="""What are the treatments for pheochromocytomas??"""

In [ ]:
import sacrebleu

reference_summaries = [
    ref_summary_1,
    ref_summary_2

]

generated_summaries_t5 = [
    summ1,
    summ2
]



# Flatten the reference summaries for sacrebleu
flattened_references = [[ref] for refs in reference_summaries for ref in refs]

# Calculate BLEU for T5
bleu_t5 = sacrebleu.corpus_bleu(generated_summaries_t5, flattened_references)
print(f"T5 BLEU score: {bleu_t5.score}")

T5 BLEU score: 3.673526562988939
